# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import jsons
import csv

#### Creating list of filepaths to process original event csv data files
1. Check your current working directory.
2. Get the current folder and subfolder event data.
3. Create a for loop to create a list of files and collect each filepath.
4. Join the file path and roots with the subdirectories using glob.

In [2]:
print(os.getcwd())


filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables
1. Initiating an empty list of rows that will be generated from each file.
2. For every filepath in the file path list.
3. Reading csv file.
4. Extracting each data row one by one and append it.
5. creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables.

In [3]:
full_data_rows_list = [] 

for f in file_path_list:


    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 

        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
     
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster
1. Make a connection to the Cassandra instance in the local machine.
2. After importing the Cluster, establish a connection and begin executing queries, need a session.

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()


session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydatabase
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydatabase')
except Exception as e:
    print(e)

# Buiding an ETL Pipeline using Apache Cassandra

### We need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

1. Create table with necessary columns(session id, item in session, artist, song, length).

2. Given that both the session id and item in session is our variables of interest, we are going to use them as the Primary keys. 

In [8]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_session(
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)      

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO song_info_by_session(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query1 = "SELECT artist, song, length FROM song_info_by_session\
                    WHERE sessionId = 338 \
                    AND itemInSession = 4"



row1 = session.execute(query1)
for row in row1:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_user(
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)
except Exception as e:
    print(e)         

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO song_info_by_user(userId,sessionId,itemInSession,artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s,%s,%s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1],line[4]))

In [13]:
query2 = " SELECT artist, song FROM song_info_by_user \
WHERE userId = 10 AND sessionId = 182 \
ORDER BY itemInSession;"


srow2 = session.execute(query2)
for row in row2:
    print(row.artist, row.song)

Down To The Bone Keep On Keepin' On
Three Drives Greece 2000
Sebastien Tellier Kilometer
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

In [14]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_name(s
            userId INT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY (song,userId));
    """)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO song_info_by_name(userId,song,firstName,lastName)"
        query = query + " VALUES (%s, %s, %s,%s);"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

In [16]:
query3 = "SELECT firstName, lastName FROM song_info_by_name WHERE song = 'All Hands Against His Own';"

row3 = session.execute(query3)
for row in row3:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("drop table song_info_by_session")
session.execute("drop table song_info_by_user")
session.execute("drop table song_info_by_name")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()